## IMPORTS

In [1]:
import re
from composable import pipeable

## DATA

In [2]:
with open('Sell Short Trades.txt') as f:
    lines = f.readlines()

## REMOVE UNWANTED LINES

In [3]:
is_blank = lambda l: len(l.strip()) == 0

Page = re.compile('^\s+\d+ of 77\s+$')
is_page_num_line = lambda l: bool(Page.match(l))

Header1 = re.compile(' Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr \n')
Header2 = re.compile(' TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr      \n')
is_header = lambda l: bool(Header1.match(l) or Header2.match(l))

Pages_gone = lambda l: is_blank(l) or is_page_num_line(l) or is_header(l)
culled_lines = [l for l in lines if not Pages_gone(l)]
culled_lines = culled_lines[3:-2]

output = ''.join(culled_lines)
with open('part2_Sell_Short_Trades_extra_removed.csv', 'w') as out_file:
    out_file.write(output)

## USING DATA WITHOUT UNWANTED LINES

In [4]:
with open('part2_Sell_Short_Trades_extra_removed.csv') as f:
    Trades = f.read()

## GROUPING LINES

In [5]:
cers_split = pipeable(lambda s: s.split(' CERS') )
cers_add = pipeable(lambda l: ['CERS'+row for row in l])

s_split = pipeable(lambda l: [row.split('\n S') for row in l])
s_add = pipeable(lambda l: ['S'+row for row in l])

s = re.compile(r'\n S')
has_s = lambda l: bool(s.search(l))


flatten = pipeable(lambda l: [item for sublist in l for item in sublist])

scers = re.compile(r'^SCERS.+')
has_scers = lambda s: bool(scers.match(s))
rid_Scers = pipeable(lambda l: [line[1:] if has_scers(line) == True else line for line in l])

rid_n = pipeable(lambda l: [row.replace('\n','') for row in l])


grouped = (Trades
>> cers_split
>> cers_add
>> s_split 
>> flatten
>> s_add
>> rid_Scers
>> rid_n
)

joined = '\n'.join(grouped[1:])
with open('part2_Sell_Short_Trades_grouped.csv', 'w') as out_file:
    out_file.write(joined)